# Data Collection and Cleanning

In this notebook, we are scraping all the data we need for our project. 

In [1]:
import requests, time, re
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
def clean_title(title):
    title = title.lower()
    title = re.sub("[(.°)]", "", title)
    title = re.sub(" ", "_", title)
    
    return title

In [3]:
def get_data(url):  
    colmuns = []
    raw_data = {}
    date = ""
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    # Go to the table containg the daily weather history & observations
    for table in soup.find_all("table", {"id": "obsTable"}):
        
        # Parse the head of the table
        for thead in table.find_all("thead"):
            for tr in thead.find_all("tr"):
                # Get column title
                for i, th in enumerate(tr.find_all("th")):
                    if i == 0:
                        date = clean_title(th.text)
                        colmuns.append(clean_title("Date"))
                    elif i in range(1, 7):
                        colmuns.append(clean_title(th.text))
                        colmuns.append(clean_title(th.text))
                        if i == 6:
                            colmuns.append(clean_title("Wind Gust (mph)"))
                        else:
                            colmuns.append(clean_title(th.text))
                    else:
                        colmuns.append(clean_title(th.text))

        # Parse the body of the table
        for j, tbody in enumerate(table.find_all("tbody")):
            for tr in tbody.find_all("tr"):
                # Get column subtitle
                if j == 0:
                    for k, td in enumerate(tr.find_all("td")):
                        if k == 0:
                            date += "/" + td.text.lower()
                        elif k != len(tr.find_all("td")) - 1:
                            colmuns[k] += "_" + td.text.lower()
                # Get data
                else:
                    for l, td in enumerate(tr.find_all("td")):
                        if colmuns[l] not in raw_data:
                            raw_data[colmuns[l]] = []

                        if l == 0:
                            raw_data[colmuns[l]].append("%s/%s" % (date, td.text))
                        elif l != len(tr.find_all("td")) - 1:    
                            raw_data[colmuns[l]].append(float(td.text.replace("\n", "").replace("-", "0")))
                        else:
                            raw_data[colmuns[l]].append(td.text.replace("\n", "").replace("\t", ""))
    
    return raw_data

In [4]:
def clean_data(df):
    new_date = df['date'].apply(lambda x: pd.Series([i for i in x.split('/')]))    
    new_date.rename(columns={0:'year',1:'month',2:'day'},inplace=True)
    
    df.drop('date', axis=1, inplace=True)
    
    return pd.concat([new_date, df], axis=1)

In [5]:
for year in range(2015, 2018):
    months = range(1, 6) if year == 2017 else range(1, 13)
    for month in months:
        data = pd.DataFrame(get_data("https://www.wunderground.com/history/airport/KSBP/%d/%d/1/MonthlyHistory.html?req_city=San Luis Obispo&req_state=CA&req_statename=California&reqdb.zip=93405&reqdb.magic=1&reqdb.wmo=99999&MR=1" % (year, month)))
        data = clean_data(data)
        
        data.to_csv("data/%d_%d.csv" % (year, month))
        
        time.sleep(0.1)

In [6]:
a = pd.read_csv("data/2017_5.csv")
data

,year,month,day,dew_point_f_avg,dew_point_f_high,dew_point_f_low,events,humidity_%_avg,humidity_%_high,humidity_%_low,...,sea_level_press_in_low,temp_f_avg,temp_f_high,temp_f_low,visibility_mi_avg,visibility_mi_high,visibility_mi_low,wind_gust_mph_high,wind_mph_avg,wind_mph_high
0,2017,may,1,50.0,55.0,45.0,,55.0,80.0,29.0,...,29.87,70.0,90.0,52.0,10.0,10.0,10.0,30.0,5.0,23.0
1,2017,may,2,52.0,57.0,48.0,,53.0,78.0,30.0,...,29.84,73.0,89.0,59.0,10.0,10.0,9.0,0.0,4.0,18.0
2,2017,may,3,54.0,59.0,51.0,Rain,64.0,81.0,42.0,...,29.88,70.0,82.0,57.0,10.0,10.0,8.0,0.0,4.0,13.0
3,2017,may,4,54.0,58.0,51.0,,78.0,93.0,52.0,...,29.91,66.0,78.0,54.0,9.0,10.0,1.0,0.0,3.0,15.0
4,2017,may,5,52.0,56.0,48.0,,80.0,93.0,61.0,...,29.83,62.0,72.0,53.0,8.0,10.0,1.0,0.0,8.0,18.0
5,2017,may,6,44.0,48.0,41.0,,70.0,83.0,48.0,...,29.75,56.0,62.0,50.0,9.0,10.0,7.0,37.0,7.0,26.0
6,2017,may,7,48.0,51.0,45.0,Rain,80.0,89.0,69.0,...,29.77,56.0,61.0,51.0,8.0,10.0,2.0,0.0,4.0,10.0
7,2017,may,8,49.0,53.0,47.0,,80.0,93.0,59.0,...,29.88,59.0,68.0,50.0,8.0,10.0,4.0,0.0,6.0,20.0
8,2017,may,9,51.0,54.0,48.0,,83.0,96.0,63.0,...,29.84,58.0,66.0,51.0,6.0,10.0,1.0,0.0,5.0,12.0
9,2017,may,10,54.0,56.0,52.0,,82.0,90.0,68.0,...,29.99,60.0,66.0,57.0,8.0,10.0,4.0,0.0,2.0,10.0


In [7]:
a

,Unnamed: 0,year,month,day,dew_point_f_avg,dew_point_f_high,dew_point_f_low,events,humidity_%_avg,humidity_%_high,...,sea_level_press_in_low,temp_f_avg,temp_f_high,temp_f_low,visibility_mi_avg,visibility_mi_high,visibility_mi_low,wind_gust_mph_high,wind_mph_avg,wind_mph_high
0,0,2017,may,1,50.0,55.0,45.0,,55.0,80.0,...,29.87,70.0,90.0,52.0,10.0,10.0,10.0,30.0,5.0,23.0
1,1,2017,may,2,52.0,57.0,48.0,,53.0,78.0,...,29.84,73.0,89.0,59.0,10.0,10.0,9.0,0.0,4.0,18.0
2,2,2017,may,3,54.0,59.0,51.0,Rain,64.0,81.0,...,29.88,70.0,82.0,57.0,10.0,10.0,8.0,0.0,4.0,13.0
3,3,2017,may,4,54.0,58.0,51.0,,78.0,93.0,...,29.91,66.0,78.0,54.0,9.0,10.0,1.0,0.0,3.0,15.0
4,4,2017,may,5,52.0,56.0,48.0,,80.0,93.0,...,29.83,62.0,72.0,53.0,8.0,10.0,1.0,0.0,8.0,18.0
5,5,2017,may,6,44.0,48.0,41.0,,70.0,83.0,...,29.75,56.0,62.0,50.0,9.0,10.0,7.0,37.0,7.0,26.0
6,6,2017,may,7,48.0,51.0,45.0,Rain,80.0,89.0,...,29.77,56.0,61.0,51.0,8.0,10.0,2.0,0.0,4.0,10.0
7,7,2017,may,8,49.0,53.0,47.0,,80.0,93.0,...,29.88,59.0,68.0,50.0,8.0,10.0,4.0,0.0,6.0,20.0
8,8,2017,may,9,51.0,54.0,48.0,,83.0,96.0,...,29.84,58.0,66.0,51.0,6.0,10.0,1.0,0.0,5.0,12.0
9,9,2017,may,10,54.0,56.0,52.0,,82.0,90.0,...,29.99,60.0,66.0,57.0,8.0,10.0,4.0,0.0,2.0,10.0
